In [51]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

from p1_graphs import *
import altair as alt
data = get_accident_data(fname="dataset_v1.csv",sample=True)
accident_data = get_weather_data(data,fname = "weather.csv")

data.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y
c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\IPython\core\interactiveshell.py:3526: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index_left,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date,weekday,covid,BoroName,index,BoroCode,Shape_Leng,Shape_Area
0,61494,2018-08-13T00:00:00Z,16:30,BRONX,40.813095,-73.907180,Suv,2018-08-13,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,82966,2018-07-08T00:00:00Z,11:20,BRONX,40.878310,-73.870155,Sedan,2018-07-08,weekend,before,Bronx,4,2,464392.991824,1.186925e+09
0,89961,2018-07-27T00:00:00Z,15:50,BRONX,40.884487,-73.865670,Pickup,2018-07-27,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,106937,2018-06-18T00:00:00Z,21:30,BRONX,40.812897,-73.901450,Suv,2018-06-18,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,76930,2018-07-27T00:00:00Z,9:30,BRONX,40.887070,-73.863396,Suv,2018-07-27,weekday,before,Bronx,4,2,464392.991824,1.186925e+09


In [104]:
mapa = get_map()
ny_df, bur = get_buroughs(mapa)
# normalize bur and accident data BoroName
bur = bur.reset_index()
accident_data.head()
# create properties.name column equal to BoroName
accident_data['properties.name'] = accident_data['BoroName']

c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y


In [112]:
accident_data.head()

,index_left,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date,weekday,covid,BoroName,index,BoroCode,Shape_Leng,Shape_Area,datetime,conditions,properties.name,properties_name
0,61494,2018-08-13T00:00:00Z,16:30,BRONX,40.813095,-73.90718,Suv,2018-08-13,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-13,"Rain, Overcast",Bronx,Bronx
1,73207,2018-08-13T00:00:00Z,13:50,NaN,40.835690,-73.86857,Pickup,2018-08-13,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-13,"Rain, Overcast",Bronx,Bronx
2,69708,2018-08-13T00:00:00Z,19:48,BRONX,40.888554,-73.90375,Suv,2018-08-13,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-13,"Rain, Overcast",Bronx,Bronx
3,61547,2018-08-13T00:00:00Z,16:20,BROOKLYN,40.668747,-73.93038,Suv,2018-08-13,weekday,before,Brooklyn,2,3,741080.523166,1.937479e+09,2018-08-13,"Rain, Overcast",Brooklyn,Brooklyn
4,71201,2018-08-13T00:00:00Z,17:00,BROOKLYN,40.633087,-73.94379,Suv,2018-08-13,weekday,before,Brooklyn,2,3,741080.523166,1.937479e+09,2018-08-13,"Rain, Overcast",Brooklyn,Brooklyn


In [123]:

selection_buro = alt.selection_point(fields=['properties.name'], empty='all')
base=alt.Chart(data_geojson_remote).mark_geoshape().properties(
    width=500,
    height=300,
).project(
    type="albersUsa"
).add_params(selection_buro).encode(
    opacity=alt.condition(selection_buro, alt.value(.8), alt.value(0.2))
)
points = alt.Chart(accident_data).mark_circle().encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    size=alt.value(2),
    color=alt.value('red'),
    opacity=alt.condition(selection_buro, alt.value(1), alt.value(0))
).add_params(selection_buro)

bar_chart = alt.Chart(accident_data).mark_bar().encode(
    x = alt.X('count()'),
    y = alt.Y('properties\\.name'),
    opacity=alt.condition(selection_buro, alt.value(1), alt.value(0.4))
)
(base+points) | bar_chart

c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


alt.HConcatChart(...)

In [129]:

colors = {"bg": "#eff0f3", "col1": "#d8b365", "col2": "#5ab4ac"}
w = 600
h = 400
ratio = 0.2
# accident_data = get_weather_data(data,fname = "weather.csv")
print(accident_data.columns)
selection_cond = alt.selection_multi(on="click", empty="all", fields=["conditions"])
# choropleth, _ = plot_map(hex_data, mapa, ny_df, bur,selection_buro,selection_cond)



ny = "https://raw.githubusercontent.com/pauamargant/VI_P1/main/resources/new-york-city-boroughs.geojson"
data_geojson_remote = alt.Data(url=ny,format=alt.DataFormat(property='features',type='json'))
selection_buro = alt.selection_point(fields=["properties.name"], empty="all")
base = (
    alt.Chart(data_geojson_remote)
    .mark_geoshape()
    .properties(
        width=500,
        height=300,
    )
    .project(type="albersUsa")
    .add_params(selection_buro)
    .encode(opacity=alt.condition(selection_buro, alt.value(0.8), alt.value(0.2)))
)
points = (
    alt.Chart(accident_data)
    .mark_circle()
    .transform_filter(selection_cond)
    .encode(
        longitude="LONGITUDE:Q",
        latitude="LATITUDE:Q",
        size=alt.value(2),
        color=alt.value("red"),
        opacity=alt.condition(selection_buro, alt.value(1), alt.value(0)),
    )
    .add_params(selection_buro)
)

bar_chart = (
    alt.Chart(accident_data)
    .mark_bar()
    .transform_filter(selection_cond)
    .encode(
        x=alt.X("count()"),
        y=alt.Y("properties\\.name"),
        opacity=alt.condition(selection_buro, alt.value(1), alt.value(0.4)),
    )
)
geo_view = (base + points) | bar_chart


bars = (
    alt.Chart(accident_data)
    .transform_filter(selection_buro)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .mark_bar(height=3, orient="horizontal")
    .encode(
        y=alt.Y("conditions:N").sort("x"),
        x="min(diff):Q",
        color=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        fill=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * (1 - ratio), height=h)
    .add_params(selection_cond)
)
dots = (
    alt.Chart(accident_data)
    .transform_filter(selection_buro)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .mark_point(height=3, orient="horizontal", size=100)
    .encode(
        y=alt.Y("conditions:N").sort("x"),
        x="min(diff):Q",
        color=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        fill=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * (1 - ratio), height=h)
    .add_params(selection_cond)
)
bar_legend = (
    alt.Chart(accident_data)
    .mark_rect()
    .transform_filter(selection_buro)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .encode(
        y=alt.Y("conditions:N", sort=alt.EncodingSortField(field="diff")),
        color="count()",
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * ratio, height=h)
    .add_params(selection_cond)
)

plot = geo_view & (bars + dots | bar_legend)

Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE',
       'LONGITUDE', 'VEHICLE TYPE CODE 1', 'date', 'weekday', 'covid',
       'BoroName', 'index', 'BoroCode', 'Shape_Leng', 'Shape_Area', 'datetime',
       'conditions', 'properties.name', 'properties_name'],
      dtype='object')


c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


alt.VConcatChart(...)

In [60]:
map

SchemaValidationError: `LayerChart` has no parameter named 'opacity'

Existing parameter names are:
layer        data          height    projection   usermeta   
autosize     datasets      name      resolve      view       
background   description   padding   title        width      
config       encoding      params    transform               

See the help for `LayerChart` to read the full description of these parameters

alt.LayerChart(...)